# Team information
|S. No|Name|email|
|-|-|-|
|1|Mathews Roy|psymr3@nottingham.ac.uk|
|2|Ewan Ross|psyer1@nottingham.ac.uk|
|3|Soham Talukdar|ppxst3@nottingham.ac.uk|
|4|Srushanth Baride|ppxsb5@nottingham.ac.uk|

# Resources used
[pyspark: Extracting, transforming and selecting features](https://spark.apache.org/docs/latest/ml-features)</br>
[sklearn mutual_info_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html)</br>
[sklearn chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html?highlight=chi2#sklearn.feature_selection.chi2)

# Data processing & plot libraries

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn import metrics
from tabulate import tabulate
from datetime import timedelta
import matplotlib.pyplot as plt

# pyspark ML libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.feature import ChiSqSelector, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier as pyspark_DecisionTreeClassifier

# sklearn ML libraries

In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.tree import DecisionTreeClassifier as sklearn_DecisionTreeClassifier

In [4]:
# Creating a new spark session
spark = SparkSession.builder.master("local[*]").appName("Feature Selection Master").getOrCreate()

In [5]:
# Path to the Leukemia csv file
_leukemia_dataset_file = "../Datasets/Leukemia_GSE9476.csv"

In [6]:
'''
Read data from Leukemia csv file
Max columns are set to 22285
Header is set to True as the csv file contains a header
'''
sparkDF = spark.read.option("maxColumns", 22285).csv(_leukemia_dataset_file, header=True)

In [7]:
# Converting spark DataFrame to pandas DataFrame for easy processing
pandasDF = sparkDF.toPandas()
pandasDF.head()

C:\ProgramData\Anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,samples,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Hs28SrRNA-5_at,AFFX-r2-Hs28SrRNA-M_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,1,Bone_Marrow_CD34,7.74524500119505,7.81121007534134,6.47791635228852,8.84150572057594,4.5469405821895,7.95771415034381,5.34499901190722,4.67336385511613,...,5.05884906355157,6.81000356185228,12.8000601994153,12.7186115474035,5.39151157002182,4.66616566659989,3.97475949012144,3.65669311996003,4.16062155445183,4.13924884553532
1,12,Bone_Marrow_CD34,8.08725200850986,7.24067331083678,8.58464840530855,8.9835712260236,4.54893416449282,8.01165205655946,5.57964654533655,4.82818367944698,...,4.43615259371806,6.75147094561877,12.4727057814784,12.3335932023584,5.37973825546594,4.65678572080881,4.18834812377709,3.79253460224534,4.2044138400184,4.12269983763514
2,13,Bone_Marrow_CD34,7.79205557755844,7.54936775520293,11.053504051356,8.90970256236097,4.54932764667886,8.23709920681109,5.40648931670832,4.61557197116622,...,4.39206061104228,6.08629542334857,12.6373840031888,12.4990375684941,5.31660434964732,4.60056619131529,3.84556141253046,3.63571499403405,4.17419911761064,4.06715189177125
3,14,Bone_Marrow_CD34,7.76726466863097,7.09446004508613,11.8164333464643,8.9946544133444,4.697018225266,8.28341179301214,5.58219459657822,4.90368448782678,...,4.63333402111035,6.37599124562392,12.9036297504518,12.8714539953803,5.17995144927903,4.64195164611807,3.99163406360053,3.70458705244103,4.14993831446596,3.9101497118226
4,15,Bone_Marrow_CD34,8.01011677792405,7.40528111947315,6.65604868129464,9.05068187803522,4.51498576534719,8.37704633235136,5.493713480124,4.86075384755779,...,5.30519187665136,6.70045349076342,12.9493522755966,12.7825147956718,5.34168947217255,4.56031502631325,3.88702024770358,3.62985266486149,4.12751339464474,4.00431605125524


In [8]:
X = sparkDF.drop('type')
y = sparkDF.select('type')

In [9]:
X_rows = np.array(X.collect()).astype(float)
y_rows = np.array(y.collect()).astype(str)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Srush\AppData\Local\Temp/ipykernel_20328/1849327092.py", line 1, in <module>
    X_rows = np.array(X.collect()).astype(float)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\sql\dataframe.py", line 693, in collect
    sock_info = self._jdf.collectToPython()
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exce

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
X_rows = preprocessing.scale(np.array(X_rows, dtype=object))

In [ ]:
'''
Function: isNaN
INPUT:
------
num: any type

OUTPUT:
-------
1. boolean value (true if null)
'''

def isNaN(num):
    return num != num

In [ ]:
valid_X = []
valid_y = []

rowID = 0

for instance in X_rows:
  valid = True
  rowID += 1
  
  # remove all null values
  for value in instance:
    if isNaN(value):
      valid = False
      print(f"row {rowID} is invalid")
      break

  if valid:
    valid_X.append(instance)
    valid_y.append(y_rows[rowID-1])

In [ ]:
preprocessed_X = pd.DataFrame(valid_X)
preprocessed_y = pd.DataFrame(valid_y)
preprocessed_X['output'] = preprocessed_y

[Code reference](https://medium.com/analytics-vidhya/removing-outliers-understanding-how-and-what-behind-the-magic-18a78ab480ff)

In [ ]:
'''
Function: check_outliers_std
INPUT:
------
dataframe: spark dataframe
col_name. column name 

OUTPUT:
-------
1. boolean value, true if outliers exist
'''

def check_outliers_std(dataframe, col_name):
    lower_boundary, upper_boundary = determine_outlier_thresholds_std(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_boundary) | (dataframe[col_name] < lower_boundary)].any(axis=None):
        return True
    else: 
        return False

In [ ]:
'''
Function: determine_outlier_thresholds_std
INPUT:
------
dataframe: spark dataframe
col_name: column name

OUTPUT:
-------
1. lower and upper bound (float values)
'''

def determine_outlier_thresholds_std(dataframe, col_name):
    upper_boundary = dataframe[col_name].mean() + 2 * dataframe[col_name].std()
    lower_boundary = dataframe[col_name].mean() - 2 * dataframe[col_name].std()
    return lower_boundary, upper_boundary

In [ ]:
'''
Function: replace_with_thresholds_std
INPUT:
------
dataframe: spark dataframe
cols: columns
replace: boolean flag (replace values if true)

OUTPUT:
-------
1. no return values; replace outliers with lower/upper limit values
'''

def replace_with_thresholds_std(dataframe, cols, replace=False):
    total = 0
    data = []
    
    # loop through all columns
    for col_name in cols:
          
        # ignore output column
        if col_name != 'output':
            outliers_ = check_outliers_std(dataframe, col_name)
            count = None
            
            # get lower/upper limit from method
            lower_limit, upper_limit = determine_outlier_thresholds_std(dataframe, col_name)
            
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                
                if replace:
                    if lower_limit < 0:
                        # We don't want to replace with negative values.
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                        
            outliers_status = check_outliers_std(dataframe, col_name)
            data.append([outliers_, outliers_status,count, col_name, lower_limit, upper_limit])
            
            if count:
              total = total + count
            
    print(f"Replaced (Total): {total}")
            
    table = tabulate(data, headers=['Outlier (Previously)','Outliers','Count', 'Column','Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    print("Removing Outliers using 2 Standard Deviation")

In [ ]:
# turn it into a dataframe
preprocessed_X = pd.DataFrame(preprocessed_X)

#df.columns = headers
replace_with_thresholds_std(preprocessed_X, preprocessed_X.columns, True)

In [ ]:
_columns = list(pandasDF.columns)
_columns.remove("type")
_columns.append("type")

In [ ]:
preprocessed_X.set_axis(_columns, axis=1, inplace=True)

In [ ]:
# Converting string categorical to numerical
LE = LabelEncoder()
preprocessed_X['type'] = LE.fit_transform(preprocessed_X['type'])
preprocessed_X['type'] = preprocessed_X['type'].astype('int32')
pandasDF = preprocessed_X 

[Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)

In [ ]:
# Getting the DataFrame shape
# The data is suffering the 'curse of dimensionality' as the no.of features are exponentially greater than the no.of samples
pandasDF.shape

In [ ]:
bone_marrow_type = pandasDF["type"].unique()

In [ ]:
# Converting pandas DataFrame to spark DataFrame
sparkDF = spark.createDataFrame(pandasDF)

# Converting spark DataFrame columns to float
sparkDF = sparkDF.select(*(F.col(c).cast("float").alias(c) for c in sparkDF.columns))

In [ ]:
# Vectorising the spark DataFrame for easier processing
vecAssembler = VectorAssembler(inputCols=sparkDF.columns, outputCol="features")
vector_sparkDF = vecAssembler.transform(sparkDF)

In [ ]:
# Setting features split
features_mul = 4000

# spark [ChiSqSelector](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.ChiSqSelector.html)
[ChiSqSelector](https://george-jen.gitbook.io/data-science-and-apache-spark/chisqselector) stands for Chi-Squared feature selection. It operates on labeled data with categorical features. ChiSqSelector uses the Chi-Squared test of independence to decide which features to choose.

In [ ]:
'''
Function: spark_ChiSqSelector
INPUT:
------
i: Number of Features to select
_vector_sparkDF: DataFrame from which the Features will be selected

OUTPUT:
-------
1. Model accuracy
2. Execution time for feature selection, building and evaluating the model
'''

def spark_ChiSqSelector(i, _vector_sparkDF):
  # Start time for execution time
  start_time = time.monotonic()
  
  # Selecting the best i features from the entire dataset
  selector = ChiSqSelector(
    numTopFeatures=i, 
    featuresCol="features", 
    outputCol="selectedFeatures", 
    labelCol="type"
  )
  result = selector.fit(_vector_sparkDF).transform(_vector_sparkDF)
  print(f"Top {selector.getNumTopFeatures()} features selected")
  
  # Splitting the data into training & testing
  (train, test) = result.randomSplit([0.7, 0.3])
  
  # Using pyspark DecisionTreeClassifier to define and fit the ML model
  dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
  model = dt.fit(train)
  
  # Make predictions
  predictions = model.transform(test)
  
  # Evaluating the predictions
  evaluator = MulticlassClassificationEvaluator(
    labelCol="type", 
    predictionCol="prediction", 
    metricName="accuracy"
  )
  accuracy = evaluator.evaluate(predictions)
  print(f"Test accuracy = {accuracy}")
  
  # End time for execution time
  end_time = time.monotonic()
  
  # Return accuracy and execution time
  return accuracy, timedelta(seconds=end_time - start_time).total_seconds()
  

# spark [UnivariateFeatureSelector](https://spark.apache.org/docs/latest/ml-features#univariatefeatureselector)
[UnivariateFeatureSelector](https://spark.apache.org/docs/latest/ml-features#univariatefeatureselector) operates on categorical/continuous labels with categorical/continuous features. User can set featureType and labelType, and Spark will pick the score function to use based on the specified featureType and labelType.

In [ ]:
'''
Function: spark_UnivariateFeatureSelector
INPUT:
------
i: Number of Features to select
_vector_sparkDF: DataFrame from which the Features will be selected

OUTPUT:
-------
1. Model accuracy
2. Execution time for feature selection, building and evaluating the model
'''

def spark_UnivariateFeatureSelector(i, _vector_sparkDF):
  # Start time for execution time
  start_time = time.monotonic()
  
  # Selecting the best i features from the entire dataset
  selector = UnivariateFeatureSelector(
    featuresCol="features", 
    outputCol="selectedFeatures", 
    labelCol="type", 
    selectionMode="numTopFeatures"
  )
  selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(i)
  result = selector.fit(_vector_sparkDF).transform(_vector_sparkDF)

  print("UnivariateFeatureSelector output with top %d features selected using f_classif" % selector.getSelectionThreshold())
  # result.show()
  
  # Splitting the data into training & testing
  (train, test) = result.randomSplit([0.7, 0.3])
  
  # Using pyspark DecisionTreeClassifier to define and fit the ML model
  dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
  model = dt.fit(train)
  
  # Make predictions
  predictions = model.transform(test)
  
  # Evaluating the predictions
  evaluator = MulticlassClassificationEvaluator(
    labelCol="type", 
    predictionCol="prediction", 
    metricName="accuracy"
  )
  accuracy = evaluator.evaluate(predictions)
  print(f"Test accuracy = {accuracy}")
  
  # End time for execution time
  end_time = time.monotonic()
  
  # Return accuracy and execution time
  return accuracy, timedelta(seconds=end_time - start_time).total_seconds()
  

In [ ]:
spark_acc_ChiSqSelector = []
spark_time_ChiSqSelector = []
spark_acc_UnivariateFeatureSelector = []
spark_time_UnivariateFeatureSelector = []

for i in range(2, len(vector_sparkDF.columns), features_mul):
  acc, exec_time = spark_ChiSqSelector(i, vector_sparkDF)
  spark_acc_ChiSqSelector.append(acc)
  spark_time_ChiSqSelector.append(exec_time)
  
  acc, exec_time = spark_UnivariateFeatureSelector(i, vector_sparkDF)
  spark_acc_UnivariateFeatureSelector.append(acc)
  spark_time_UnivariateFeatureSelector.append(exec_time)

# sklearn [chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)
[ChiSqSelector](https://george-jen.gitbook.io/data-science-and-apache-spark/chisqselector) stands for Chi-Squared feature selection. It operates on labeled data with categorical features. ChiSqSelector uses the Chi-Squared test of independence to decide which features to choose.

In [ ]:
# X is features
X = pandasDF[pandasDF.columns.drop('type')]

# y is labels
y = pandasDF['type']

In [ ]:
'''
Function: sklearn_chi2
INPUT:
------
i: Number of Features to select
_X: Features
_y: labels

OUTPUT:
-------
1. Model accuracy
2. Execution time for feature selection, building and evaluating the model
'''

def sklearn_chi2(i, _X, _y):
  # Start time for execution time
  start_time = time.monotonic()
  
  # Selecting the best i features from the entire dataset
  X_new = SelectKBest(chi2, k=i).fit_transform(_X, _y)
  
  # Splitting the data into training & testing
  train, test, train_labels, test_labels = train_test_split(X_new, _y, test_size=0.30, random_state=42)
  
  # Using sklearn DecisionTreeClassifier to define and fit the ML model
  clf = sklearn_DecisionTreeClassifier()
  clf = clf.fit(train, train_labels)
  
  # Make predictions
  predictions = clf.predict(test)
  
  # Evaluating the predictions
  accuracy = metrics.accuracy_score(test_labels, predictions)
  print(f"Test accuracy = {accuracy}")
  
  # End time for execution time
  end_time = time.monotonic()
  
  # Return accuracy and execution time
  return accuracy, timedelta(seconds=end_time - start_time).total_seconds()

# sklearn [mutual_info_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html?highlight=mutual_info_regression#sklearn.feature_selection.mutual_info_regression)
[Estimate mutual information](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html?highlight=mutual_info_regression#sklearn.feature_selection.mutual_info_regression) for a continuous target variable.</br>
Mutual information (MI) between two random variables is a non-negative value, which measures the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.</br>
The function relies on nonparametric methods based on entropy estimation from k-nearest neighbors distances.

In [ ]:
'''
Function: sklearn_mutual_info_regression
INPUT:
------
i: Number of Features to select
_X: Features
_y: labels

OUTPUT:
-------
1. Model accuracy
2. Execution time for feature selection, building and evaluating the model
'''

def sklearn_mutual_info_regression(i, _X, _y):
  # Start time for execution time
  start_time = time.monotonic()
  
  # Selecting the best i features from the entire dataset
  selector = SelectKBest(mutual_info_regression, k=i)
  selector.fit(_X, _y)
  
  X_new = X[X.columns[selector.get_support()]]
  
  # Splitting the data into training & testing
  train, test, train_labels, test_labels = train_test_split(X_new, _y, test_size=0.30, random_state=42)
  
  # Using sklearn DecisionTreeClassifier to define and fit the ML model
  clf = sklearn_DecisionTreeClassifier()
  clf = clf.fit(train, train_labels)
  
  # Make predictions
  predictions = clf.predict(test)
  
  # Evaluating the predictions
  accuracy = metrics.accuracy_score(test_labels, predictions)
  print(f"Test accuracy = {accuracy}")
  
  # End time for execution time
  end_time = time.monotonic()
  
  # Return accuracy and execution time
  return accuracy, timedelta(seconds=end_time - start_time).total_seconds()

In [ ]:
sklearn_acc_chi2 = []
sklearn_time_chi2 = []
sklearn_acc_mutual_info_regression = []
sklearn_time_mutual_info_regression = []

for i in range(2, X.columns.size, features_mul):
  acc, exec_time = sklearn_chi2(i, X, y)
  sklearn_acc_chi2.append(acc)
  sklearn_time_chi2.append(exec_time)
  
  acc, exec_time = sklearn_mutual_info_regression(i, X, y)
  sklearn_acc_mutual_info_regression.append(acc)
  sklearn_time_mutual_info_regression.append(exec_time)

# Visualisation

In [ ]:
x_axis = list([i for i in range(2, X.columns.size, features_mul)])

In [ ]:
plt.plot(x_axis, spark_acc_ChiSqSelector, label='spark_ChiSqSelector')
plt.plot(x_axis, spark_acc_UnivariateFeatureSelector, label='spark_UnivariateFeatureSelector')
plt.plot(x_axis, sklearn_acc_chi2, label='sklearn_chi2')
plt.plot(x_axis, sklearn_acc_mutual_info_regression, label='sklearn_mutual_info_regression')
plt.ylim(0 , 1.1)
plt.title("Features vs Accuracy")
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, spark_time_ChiSqSelector, label='spark_ChiSqSelector')
plt.plot(x_axis, sklearn_time_chi2, label='sklearn_chi2')
plt.plot(x_axis, spark_time_UnivariateFeatureSelector, label='spark_UnivariateFeatureSelector')
plt.plot(x_axis, sklearn_time_mutual_info_regression, label='sklearn_mutual_info_regression')
plt.title("Features vs Execution Time")
plt.xlabel('Number of Features')
plt.ylabel('Execution Time in Seconds')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, spark_acc_ChiSqSelector, label='spark_ChiSqSelector')
plt.plot(x_axis, sklearn_acc_chi2, label='sklearn_chi2')
plt.ylim(0 , 1.1)
plt.title("Features vs Accuracy")
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, spark_time_ChiSqSelector, label='spark_ChiSqSelector')
plt.plot(x_axis, sklearn_time_chi2, label='sklearn_chi2')
plt.title("Features vs Execution Time")
plt.xlabel('Number of Features')
plt.ylabel('Execution Time in Seconds')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, spark_acc_UnivariateFeatureSelector, label='spark_UnivariateFeatureSelector')
plt.plot(x_axis, sklearn_acc_mutual_info_regression, label='sklearn_mutual_info_regression')
plt.ylim(0 , 1.1)
plt.title("Features vs Accuracy")
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, spark_time_UnivariateFeatureSelector, label='spark_UnivariateFeatureSelector')
plt.plot(x_axis, sklearn_time_mutual_info_regression, label='sklearn_mutual_info_regression')
plt.title("Features vs Execution Time")
plt.xlabel('Number of Features')
plt.ylabel('Execution Time in Seconds')
plt.legend()
plt.show()